In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_with_lag/data_with_lag.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_4"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



          Umsatz  Temp_Very_Cold_current  Temp_Very_Cold_prev  \
5457   65.890169                       0                  0.0   
5458   74.543917                       0                  0.0   
5459   69.262728                       0                  0.0   
5460   61.490175                       0                  0.0   
5461   86.759861                       0                  0.0   
...          ...                     ...                  ...   
7218   65.563734                       0                  0.0   
7219   54.047422                       0                  0.0   
7220  128.952852                       0                  0.0   
7221   71.822977                       0                  0.0   
7222   57.102795                       0                  0.0   

      Temp_Very_Cold_next  Temp_Cold_current  Temp_Cold_prev  Temp_Cold_next  \
5457                  0.0                  0             0.0             0.0   
5458                  0.0                  0             0.

In [2]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Zeilen- und Spaltenanzeige in Pandas erweitern
pd.set_option("display.max_rows", None)  # Zeigt alle Zeilen
pd.set_option("display.max_columns", None)  # Zeigt alle Spalten

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                            1.000000
Sonntag_current                   0.534262
Montag_next                       0.520507
Samstag_prev                      0.516330
Monat_2_next                      0.305718
Monat_2_current                   0.302366
Monat_2_prev                      0.295061
Heimspiel_prev                    0.275164
Feiertag_current                  0.161383
Winter_next                       0.146106
Winter_current                    0.142874
Winter_prev                       0.140161
Temp_Cold_next                    0.119371
Temp_Cold_prev                    0.089300
Temp_Cold_current                 0.087436
Werktag_next                      0.076544
Ostertag_prev                     0.066274
Monat_8_prev                      0.064655
Number_of_ships_scaled_prev       0.064571
Number_of_ships_prev              0.064571
Monat_8_current                   0.055671
Monat_8_next                      0.051115
Feiertag_next                     0.049845
Cloud_Partl

In [3]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)



Sonntag_current                   0.146606
Montag_next                       0.142722
Samstag_prev                      0.141969
Werktag_current                   0.090277
Werktag_prev                      0.074720
VPI_current                       0.060983
VPI_prev                          0.048552
VPI_next                          0.048225
Monat_2_next                      0.036402
Heimspiel_prev                    0.035349
Monat_2_current                   0.033423
Monat_2_prev                      0.031279
Dienstag_prev                     0.020886
Sommer_next                       0.016908
Monat_6_prev                      0.016725
wetter_sehr_schlecht_next         0.015795
Winter_current                    0.015704
Mittwoch_current                  0.015191
Winter_prev                       0.014791
Weather_Severe_next               0.014770
Winter_next                       0.014392
Number_of_ships_scaled_current    0.014174
Sommer_current                    0.014047
Number_of_s

In [4]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Temp_Cold_current', 'Temp_Cold_prev', 'Temp_Cold_next', 'Cloud_Cloudy_current', 'Cloud_Cloudy_prev', 'Cloud_Cloudy_next', 'Wind_Light_current', 'Wind_Light_prev', 'Wind_Light_next', 'Wind_Moderate_current', 'Wind_Moderate_prev', 'Wind_Moderate_next', 'Wind_Strong_current', 'Wind_Strong_prev', 'Wind_Strong_next', 'KielerWoche_current', 'KielerWoche_prev', 'KielerWoche_next', 'Montag_current', 'Montag_prev', 'Montag_next', 'Dienstag_current', 'Dienstag_prev', 'Dienstag_next', 'Mittwoch_current', 'Mittwoch_prev', 'Mittwoch_next', 'Donnerstag_current', 'Donnerstag_prev', 'Donnerstag_next', 'Freitag_current', 'Freitag_prev', 'Freitag_next', 'Samstag_current', 'Samstag_prev', 'Samstag_next', 'Sonntag_current', 'Sonntag_prev', 'Sonntag_next', 'VPI_current', 'VPI_prev', 'VPI_next', 'Number_of_ships_current', 'Number_of_ships_prev', 'Number_of_ships_next', 'Ship_current', 'Ship_prev', 'Ship_next', 'is_holiday_current', 'is_holiday_prev', 'is_holiday_next', 'Weihnac

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                            Feature           VIF
0            Temp_Very_Cold_current           inf
1               Temp_Very_Cold_prev           inf
2               Temp_Very_Cold_next           inf
3                 Temp_Cold_current           inf
4                    Temp_Cold_prev           inf
5                    Temp_Cold_next           inf
6                 Temp_Mild_current           inf
7                    Temp_Mild_prev           inf
8                    Temp_Mild_next           inf
9                 Temp_Warm_current           inf
10                   Temp_Warm_prev           inf
11                   Temp_Warm_next           inf
12                 Temp_Hot_current           inf
13                    Temp_Hot_prev           inf
14                    Temp_Hot_next  3.170404e+07
15              Cloud_Clear_current  6.281500e+04
16                 Cloud_Clear_prev  1.355748e+05
17                 Cloud_Clear_next  3.720711e+04
18      Cloud_Partly_Cloudy_current  1.192650e+06


In [6]:
# suche miteinander korrelierrende features

In [7]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Monat_1_next', 'Monat_1_prev'), ('Winter_prev', 'Temp_Mild_prev'), ('Monat_7_prev', 'Sommer_next'), ('Winter_next', 'Temp_Mild_prev'), ('Monat_8_next', 'Sommer_next'), ('Monat_1_prev', 'Winter_current'), ('Cloud_ok_next', 'Cloud_Partly_Cloudy_next'), ('Monat_5_next', 'Monat_5_current'), ('Monat_11_prev', 'Herbst_current'), ('Ship_next', 'Number_of_ships_next'), ('Sommer_prev', 'Temp_Warm_next'), ('Cloud_Cloudy_current', 'Cloud_Partly_Cloudy_current'), ('Monat_2_next', 'Winter_next'), ('Monat_12_next', 'Weihnachtsmarkt_next'), ('zwischen_den_jahren_next', 'zwischen_den_jahren_prev'), ('Monat_4_next', 'Monat_4_current'), ('Wind_ok_prev', 'Wind_Strong_prev'), ('Winter_current', 'Temp_Mild_next'), ('Monat_6_next', 'KielerWoche_prev'), ('Werktag_current', 'Samstag_prev'), ('Werktag_current', 'Samstag_current'), ('Sommer_prev', 'Temp_Warm_current'), ('Samstag_next', 'Donnerstag_prev'), ('Samstag_next', 'Freitag_current'), ('wetter_sehr_schön_prev', 'Temp_H